In [1]:
import numpy as np
# from fastai import *
import csv
from glob import glob
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import torch
import fastai
from fastai.basics import *
from fastai.vision.all import *
from fastai.callback.schedule import lr_find
from torch import nn
import torch.nn.functional as F
from model_fastai import test_model
# from tqdm import tqdm
from tqdm.notebook import trange, tqdm

In [6]:
df = pd.read_csv('output.csv', error_bad_lines=False)

In [7]:
X = list(df['image'])

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, X, test_size=0.33, random_state=42)

In [9]:
tmp = np.load(X[0], allow_pickle=True)
img = tmp['x']
img = normalize(img, norm='max')
print(img.mean())
print(img.std())
print(img.var())
# img = np.reshape(img,(1,img.shape[0], img.shape[1]))
# img = torch.from_numpy(img).float()
# print(img.shape)
# imgs = get_image_files(X)

0.0012281887
0.09340895
0.008725232


In [44]:
train_model = test_model()
train_model = train_model.cuda()

In [45]:
epochs = 20
batch_size = 8

In [46]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [47]:
class data_gen(torch.utils.data.Dataset):
    def __init__(self, files):
        
        self.files = files
        
        
    def __getitem__(self, i):
        
        file1 = self.files[i]
        
        tmp = np.load(file1, allow_pickle=True)
        
        img = tmp['x']
        img = normalize(img, norm='max')
        img = np.reshape(img,(1,img.shape[0], img.shape[1]))
        img = torch.from_numpy(img).float()
        
        return img

    def __len__(self): 
        
        return len(self.files)

In [48]:
optimizer = torch.optim.Adam(train_model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

In [49]:
train_dataloader = data_gen(X_train)
train_loader = torch.utils.data.DataLoader(
    train_dataloader, batch_size=8, shuffle=True, num_workers=4, pin_memory=True
)

In [50]:
valid_dataloader = data_gen(X_test)
valid_loader = torch.utils.data.DataLoader(
    valid_dataloader, batch_size=8, shuffle=False, num_workers=4
)

In [51]:
# count=0
# for j in tqdm(train_loader):
#     count+=1
#     print(j.shape)
# print(count)

In [52]:
for epoch in range(epochs):
    loss = 0
    for x_train in tqdm(train_loader):
        # reshape mini-batch data to [N, 784] matrix
        # load it to the active device
#         batch_features = batch_features.view(-1, 784).to(device)
#         print("DONE cuda")
        x_train = x_train.cuda()
        
        # reset the gradients back to zero
        # PyTorch accumulates gradients on subsequent backward passes
        optimizer.zero_grad()
        
        # compute reconstructions
        outputs = train_model(x_train)
            
        # compute training reconstruction loss
        train_loss = criterion(outputs, x_train)
        
#         # VALIDATION LOSS
#         train_model.eval()
#         for valid in valid_loader:
        
        # compute accumulated gradients
        train_loss.backward()
        
        # perform parameter update based on current gradients
        optimizer.step()
        
        # add the mini-batch training loss to epoch loss
        loss += train_loss.item()
    
    # compute the epoch training loss
    loss = loss / len(train_loader)
    
    # display the epoch training loss
    print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, epochs, loss))


epoch : 1/20, loss = 0.043382



epoch : 2/20, loss = 0.009586



epoch : 3/20, loss = 0.009272



epoch : 4/20, loss = 0.009212



epoch : 5/20, loss = 0.009186


...


epoch : 16/20, loss = 0.008854



epoch : 17/20, loss = 0.009263



epoch : 18/20, loss = 0.008831



epoch : 19/20, loss = 0.008826



epoch : 20/20, loss = 0.008822


In [53]:
save_folder = '/media/sarvagya/66e1ccd6-9d4b-4a61-8750-36e2e23d8988/solar/models/'
torch.save(train_model.state_dict(), save_folder+'model_pytorch_max.pt')